## Data Visualisation and Exploration of the Iceland Stake Measurements Dataset

@Author: Julian Biesheuvel  
Email: j.p.biesheuvel@student.tudelft.nl  
Date Created: 04/06/2024

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib import rcParams
from matplotlib import gridspec

rcParams['axes.titlepad'] = 20 

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

PLOTTING = True

In [2]:
# Load the datasets of Iceland and Norway containing the stake measurements, topographical and climatological features. 
file_dir = '.././data/'

df_iceland = pd.read_csv(file_dir + 'files/Iceland_Stake_Data_Cleaned.csv')
df_norway = pd.read_csv(file_dir + 'files/Norway_Stake_Data.csv', index_col=False)

# Change the column names for the mass balance similar between the two datasets for convenience during plotting
df_norway['elevation'] = df_norway['altitude'].astype(int)
df_norway['bw_stratigraphic'] = df_norway['balance_winter']
df_norway['bs_stratigraphic'] = df_norway['balance_summer']
df_norway['ba_stratigraphic'] = df_norway['balance_netto']

### Basic Statistics of the Datasets of Iceland and Norway

In [3]:
df_iceland.describe()

In [4]:
df_norway.describe()

### Overview of Annual Glacier Measurements in Iceland

In [5]:
# Define the range of years and unique glacier IDs
measurement_years = np.arange(df_iceland['yr'].min(), df_iceland['yr'].max() + 1)
glacier_ids = df_iceland.dropna(subset=['RGIId'])['RGIId'].unique()

# Create DataFrame to store number of measurements per year per glacier
num_measurements = pd.DataFrame(index=glacier_ids, columns=measurement_years).fillna(0)

# Calculate the number of measurements for each glacier and year
for idx in num_measurements.index:
    temp = df_iceland.loc[df_iceland['RGIId'] == idx].groupby('yr')['stake'].count()
    num_measurements.loc[idx, temp.index] = temp

# Plotting
plt.figure(figsize=(16, 8))
heatmap = sns.heatmap(
    data=num_measurements, 
    cmap='GnBu', 
    alpha=0.8, 
    cbar_kws={'label': 'Number of Measurements', 'pad': 0.005}, 
    annot=True, 
    annot_kws={'alpha': 0.9, 'fontsize': 10}, 
    linewidths=.5
)

# Customize plot appearance
plt.xlabel('Year', fontsize=12)
plt.ylabel('Glacier ID', fontsize=12)
plt.title('Number of Stake Measurements per Glacier per Year', fontsize=14)
plt.xticks(np.arange(len(measurement_years)) + 0.5, measurement_years, rotation=45, fontsize=10)
plt.yticks(np.arange(len(glacier_ids)) + 0.5, glacier_ids, rotation=0, fontsize=10)
plt.grid(False)
plt.tight_layout()

# Save the plot if PLOTTING is defined and True
if 'PLOTTING' in globals() and PLOTTING:
    plt.savefig(file_dir + 'plots/num_measurements_per_year_per_glacier.svg', dpi=600, format='svg')

plt.show()
plt.close()

### Stacked Bar Chart of Annual, Winter, and Summer Surface Mass Balance Measurements per Year

In [6]:
# Group measurements by year for each type
bw_num_measurements = df_iceland.groupby('yr')['bw_stratigraphic'].count()
bs_num_measurements = df_iceland.groupby('yr')['bs_stratigraphic'].count()
ba_num_measurements = df_iceland.groupby('yr')['ba_stratigraphic'].count()

# Plotting
plt.figure(figsize=(16, 6))

# Plot Winter measurements
plt.bar(
    bw_num_measurements.index, 
    bw_num_measurements.values, 
    label='Winter', 
    color='#4e8cd9', 
    alpha=0.7
)

# Plot Summer measurements on top of Winter measurements
plt.bar(
    bs_num_measurements.index, 
    bs_num_measurements.values, 
    bottom=bw_num_measurements.values, 
    label='Summer', 
    color='#9c4b75', 
    alpha=0.7
)

# Plot Annual measurements on top of Winter and Summer measurements
plt.bar(
    ba_num_measurements.index, 
    ba_num_measurements.values, 
    bottom=bw_num_measurements.values + bs_num_measurements.values, 
    label='Annual', 
    color='#4b9c8d', 
    alpha=0.7
)

# Adding labels and title
plt.xlabel('Year', fontsize=15)
plt.ylabel('Number of Measurements', fontsize=15)
plt.title('Total Number of Seasonal and Annual Measurements per Year', fontsize=17)

# Setting tick parameters
plt.xticks(ticks=bs_num_measurements.index, rotation=45, fontsize=12)
plt.yticks(fontsize=12)

# Centering x-axis labels
ax = plt.gca()
plt.setp(ax.xaxis.get_majorticklabels(), ha='center')

# Adding legend
plt.legend(fontsize=14)

# Adjusting layout and grid
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', linewidth=0.7)
plt.gca().xaxis.grid(False)

# Save the plot if PLOTTING is defined and True
if 'PLOTTING' in globals() and PLOTTING:
    plt.savefig(file_dir + '/plots/num_measurements_per_year.svg', dpi=600, format='svg')

plt.show()
plt.close()

### Create a 2D Histogram of the Three Key Topographical Features of Icelandic Glaciers Versus Annual Mass Balance

In [7]:
def create_joint_grid(df, y_var, title, y_label, log=False):
    """
    Create a joint grid plot showing the relationship between annual mass balance and a given variable.

    Parameters:
        df (DataFrame): The DataFrame containing the data.
        y_var (str): The name of the variable to be plotted against annual mass balance.
        title (str): The title of the plot.
        y_label (str): The label for the y-axis.
        log (bool, optional): Whether to use a logarithmic scale on the y-axis. Defaults to False.
    """
    sns.set_theme(style="ticks")
    
    # Initialize the JointGrid
    g = sns.JointGrid(data=df, x='ba_stratigraphic', y=y_var, marginal_ticks=True)
    
    # Create an inset legend for the histogram colorbar
    cax = g.figure.add_axes([.15, .55, .02, .25])
    
    g.ax_joint.grid(True)
    
    if log:
        g.ax_joint.set(yscale="log")
    
    # Add the joint and marginal histogram plots
    g.plot_joint(
        sns.histplot, discrete=(True, False),
        cmap="light:#4e8cd9", pmax=.8, cbar=True, cbar_ax=cax
    )
    g.plot_marginals(sns.histplot, color="#4e8cd9", linewidth=.5)
    
    plt.suptitle(f"Annual Surface Mass Balance by {title}", fontsize=14, y=1)
    
    g.set_axis_labels("Annual Surface Mass Balance [m w.e.]", y_label)
    
    if 'PLOTTING' in globals() and PLOTTING:
        plt.savefig(file_dir + f'plots/{title}.svg', dpi=600, format='svg')
    plt.show()
    plt.close()

In [8]:
create_joint_grid(df_iceland, "elevation", "Elevation", "Elevation [m]")

In [9]:
create_joint_grid(df_iceland, "slope", "Slope", "Slope [Rad]", True)

In [10]:
create_joint_grid(df_iceland, "aspect", "Aspect", "Aspect [Rad]", False)

### Create a Correlation Matrix for Topographical and Climatological Features with Mass Balances

In [11]:
# List of suffixes for summer and winter months
summer_suffixes = ['_apr', '_may', '_jun', '_jul', '_aug', '_sep']
winter_suffixes = ['_oct', '_nov', '_dec', '_jan', '_feb', '_mar']

def calculate_seasonal_average(df):
    """
    Filters the DataFrame columns based on specified suffixes and calculates averages for summer and winter chunks.

    Parameters:
        df (DataFrame): The DataFrame containing the data.

    Returns:
        tuple: A tuple containing two DataFrames:
               - summer_avg: DataFrame with summer averages and additional columns.
               - winter_avg: DataFrame with winter averages and additional columns.
    """
    # Selecting columns based on the suffixes
    summer_columns = [col for col in df.columns if col.endswith(tuple(summer_suffixes))]
    winter_columns = [col for col in df.columns if col.endswith(tuple(winter_suffixes))]
    
    # Creating separate DataFrames for summer and winter columns
    summer_cols = df[summer_columns]
    winter_cols = df[winter_columns]
    
    # Calculating averages for summer and winter columns
    summer_avg = calculate_average_for_season(summer_cols)
    winter_avg = calculate_average_for_season(winter_cols)
    
    # Adding additional columns to summer and winter DataFrames
    additional_columns = ['bw_stratigraphic', 'bs_stratigraphic', 'ba_stratigraphic']
    summer_avg[additional_columns] = df[additional_columns]
    winter_avg[additional_columns] = df[additional_columns]
    
    return summer_avg, winter_avg

def calculate_average_for_season(df, chunk_size=6):
    """
    Calculate the average for every chunk of columns, the average of each climate variable for each season, return this.

    Parameters:
        df (DataFrame): The DataFrame containing the climate variables.
        chunk_size (int): The size of each chunk of columns.

    Returns:
        DataFrame: A new DataFrame containing the average value for each season, of each climate variable.
    """
    # Reshape the DataFrame into chunks
    chunks = np.array_split(df.values, -(-len(df.columns) // chunk_size), axis=1)

    columns = [col_name[:-4] for col_name in df.columns.values]
    columns = np.flip(np.unique(columns))
    
    # Calculate averages for each chunk and concatenate them
    avg_chunks = [np.mean(chunk, axis=1) for chunk in chunks]
    avg_df = pd.DataFrame(np.column_stack(avg_chunks), columns=columns) 
    
    return avg_df

In [12]:
# List of columns to keep
topo_columns = ['dis_from_border', 'slope', 'aspect', 'elevation', 'lon', 'lat', 'bw_stratigraphic','bs_stratigraphic', 'ba_stratigraphic']
topo_ticks_labels = ['Distance\nfrom Border', 'Slope', 'Aspect', 'Elevation', 'Longitude', 'Latitude']
mbs_tick_labels= ['Winter SMB', 'Summer SMB','Annual SMB']

# Copy the DataFrame and keep only the specified columns for the topographical features
df_subset_iceland_topo = df_iceland[topo_columns].copy()
df_subset_norway_topo = df_norway[topo_columns].copy()

# Copy the DataFrame and keep only the specified columns for the climatological features
df_subset_iceland_climate = df_iceland.copy()
df_subset_norway_climate = df_norway.copy()

# Calculate the average seasonal value for the 22 climate variables, for both Iceland and Norway
summer_avg_IS, winter_avg_IS = calculate_seasonal_average(df_subset_iceland_climate)
summer_avg_NR, winter_avg_NR = calculate_seasonal_average(df_subset_norway_climate)

In [34]:
def plot_correlation_matrix(df1, df2, ticks_labels, suptitle, ax1_title, ax2_title, f_name, type_plot):
    if type_plot == 'topo':
        fig = plt.figure(figsize=(7, 11))
        gs = gridspec.GridSpec(3, 1, height_ratios=[1, 1, 0.10], width_ratios=[1], hspace=.5)
    else:
        fig = plt.figure(figsize=(10, 7))
        gs = gridspec.GridSpec(3, 1, height_ratios=[1, 1, 0.10], width_ratios=[1], hspace=0.1)

    # Create subplots within the gridspec
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])
    cax = fig.add_subplot(gs[2])

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(20, 230, as_cmap=True)

    variables_of_interest = ['bw_stratigraphic', 'bs_stratigraphic', 'ba_stratigraphic']
    correlation_subset_df1 = df1.corr().loc[variables_of_interest].iloc[:, :-3]
    correlation_subset_df2 = df2.corr().loc[variables_of_interest].iloc[:, :-3]

    sns.heatmap(correlation_subset_df1, ax=ax1, cmap=cmap, cbar=False, vmin=-1, vmax=1, center=0,
                square=True, linewidths=.5, yticklabels=mbs_tick_labels, xticklabels=ticks_labels)
    sns.heatmap(correlation_subset_df2, ax=ax2, cmap=cmap, cbar=False, vmin=-1, vmax=1, center=0,
                square=True, linewidths=.5, yticklabels=mbs_tick_labels, xticklabels=ticks_labels)

    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)
    
    ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
    ax2.set_yticklabels(ax2.get_yticklabels(), rotation=0)

    plt.setp(ax1.xaxis.get_majorticklabels(), ha='right')
    plt.setp(ax2.xaxis.get_majorticklabels(), ha='right')

    ax1.set_title(ax1_title, fontsize=14)
    ax2.set_title(ax2_title, fontsize=14)
    fig.suptitle(suptitle, fontsize=16, y=.97)

    # Add colorbar at the bottom of the plots
    cb = fig.colorbar(ax2.get_children()[0], cax=cax, orientation='horizontal', location='bottom')
    cb.outline.set_visible(False)
    cb.ax.set_xlabel('Pearson Correlation')
    cax.set_aspect('auto')

    if type_plot == 'topo':
        plt.tight_layout(h_pad=1)
    else:
        plt.tight_layout(h_pad=0)

    if 'PLOTTING' in globals() and PLOTTING:
        plt.savefig(f_name, dpi=600, format='svg', bbox_inches='tight')

    plt.show()
    plt.close(fig)

In [35]:
# Iceland - Topographical Features
plot_correlation_matrix(
    df_subset_iceland_topo,
    df_subset_norway_topo,
    topo_ticks_labels,
    'Pearson Correlation Matrix for Topographical Features\nand Surface Mass Balances',
    'Iceland',
    'Norway',
    file_dir + '/plots/corr_topo.svg',
    'topo'
)

In [36]:
# Iceland - Climate - Averaged Seasonal
climate_ticks_labels = [i for i in summer_avg_IS.columns[:-3]]

plot_correlation_matrix(
    summer_avg_IS, 
    winter_avg_IS,
    climate_ticks_labels,
    'Pearson Correlation Matrix for Averaged Seasonal Climatological Features\nand Surface Mass Balances',
    'Iceland - Summer',
    'Iceland - Winter',
    file_dir + 'plots/corr_climate_IS.svg',
    'climate'
)

In [16]:
plot_correlation_matrix(
    summer_avg_NR, 
    winter_avg_NR,
    climate_ticks_labels,
    'Pearson Correlation Matrix for Averaged Seasonal Climatological Features\nand Surface Mass Balances',
    'Norway - Summer',
    'Norway - Winter',
    file_dir + 'plots/corr_climate_NR.svg',
    'climate'
)

### Create Histograms of Seasonal and Annual Mass Balances along with Topographical Aspects for Glaciers in Iceland and Norway


In [17]:
def plot_dual_histograms(columns, axs_titles, x_labels, binwidths, titleplot, df1 = df_iceland, df2 = df_norway):
    """
    Plot dual histograms comparing two datasets across multiple columns.
    
    Parameters:
        df1 (DataFrame): The first dataset for comparison. Default is df_iceland.
        df2 (DataFrame): The second dataset for comparison. Default is df_norway.
        columns (list): List of column names to plot histograms for.
        axs_titles (list): Titles for each subplot.
        x_labels (list): Labels for the x-axis of each subplot.
        binwidths (list): Bin widths for histograms.
        titleplot (str): Title for the entire plot.
    
    Returns:
        None
    """
    
    # Define height ratios for subplots
    height_ratios = [1, 5]

    # Create subplots with defined height ratios
    fig, axs = plt.subplots(2, 3, figsize=(16, 6), gridspec_kw={'height_ratios': height_ratios})
    axs = axs.T  # Transpose axes for convenient iteration

    # Iterate over columns and corresponding subplot parameters
    for ax, data, xlabel, title, binwidth in zip(axs, columns, x_labels, axs_titles, binwidths):
        # Boxplot of concatenated data
        sns.boxplot(data=pd.concat([df1[data], df2[data]], axis=1, keys=['Iceland', 'Norway']), orient="h", ax=ax[0], palette=['#4e8cd9', '#4b9c8d'])

        # Histograms for each dataset
        sns.histplot(df1[data], ax=ax[1], binwidth=binwidth, color='#4e8cd9', alpha=0.5, label='Iceland')
        sns.histplot(df2[data], ax=ax[1], binwidth=binwidth, color='#4b9c8d', alpha=0.5, label='Norway')

        # Set xlabel, title, and tick parameters for histogram subplot
        ax[1].set_xlabel(xlabel, fontsize=14)
        ax[1].set_title(title, fontsize=17)
        ax[1].tick_params(axis='x', labelsize=12)

        # Disable y-axis labels for all but the first subplot
        if ax[1] != axs[0][1]:
            ax[1].set_ylabel(None)
        ax[1].tick_params(axis='y', labelsize=12)

    # Create legend handles and labels
    labels_handles = {label: handle for ax in fig.axes for handle, label in zip(*ax.get_legend_handles_labels())}

    # Place legend at the bottom horizontally
    plt.figlegend(
        labels_handles.values(),
        labels_handles.keys(),
        loc='lower center',
        ncol=2,
        labelspacing=0.,
        bbox_to_anchor=(0, -.05, 1, 1),
        bbox_transform=plt.gcf().transFigure
    )

    # Set main title and adjust layout
    plt.suptitle(titleplot, fontsize=17)
    plt.tight_layout()

    # Save or show the plot based on savefig parameter
    if 'PLOTTING' in globals() and PLOTTING:
        plt.savefig(file_dir + f'plots/{titleplot}.svg', dpi=600, format='svg', bbox_inches='tight')
        
    plt.show()
    plt.close(fig)

In [37]:
cols_topo = ['elevation', 'slope', 'aspect']
labels_topo = ['Elevation [m] a.s.l.', 'Slope [Rad]', 'Aspect [Rad]']
titles_topo = ['No. Points by Elevation', 'No. Points by Slope', 'No. Points by Aspect']
fig_title_topo = '3 most Important Topographical Features Distributions of Glacier in Iceland and Norway'
binwidths_topo = [150, 0.05, 0.5]

plot_dual_histograms(cols_topo, titles_topo, labels_topo, binwidths_topo, fig_title_topo)

In [19]:
cols_mass_balance = ['bw_stratigraphic', 'bs_stratigraphic', 'ba_stratigraphic']
labels_mass_balance = ['Surface Mass Balance [m w.e.]'] * 3
titles_mass_balance = ['Freq. of Winter SMB', 'Freq. of Summer SMB', 'Freq. of Annual SMB']
fig_title_mass_balance = 'Seasonal and Annual Surface Mass Balance Distributions of Glaciers in Iceland and Norway'
binwidths_mass_balance = [.6, 1, 1]

plot_dual_histograms(cols_mass_balance, titles_mass_balance, labels_mass_balance, binwidths_mass_balance, fig_title_mass_balance)

### Create a Time Series for All Stake Data in Iceland

In [20]:
# Load the climate data and get the mean t2m (temp. 2m above ground level), and average it per year and per area (longitude and latittude)
with xr.open_dataset(file_dir + 'climate/ERA5_monthly_averaged_climate_data.nc') as ds_climate:
    ds = ds_climate.t2m.resample(time="1y").mean(dim=["time", "latitude", "longitude"])
    ds = ds.reduce(np.nansum, 'expver') 
    temps = ds.values - 273.15  # Convert to Celsius directly
    temps = temps[:-1]  # Remove the last element
    
df_iceland_amb = df_iceland[['yr', 'stake', 'ba_stratigraphic']]

# Calculate the mean and standard deviation of the annual mass balance for all stake data in Iceland
mean_ba_per_year = df_iceland_amb.groupby('yr')['ba_stratigraphic'].mean()
std_ba_per_year = df_iceland_amb.groupby('yr')['ba_stratigraphic'].std()

In [21]:
def consecutive_nan(vals):
    # Calculate the difference between consecutive years
    year_diff = np.diff(vals)

    # Check if there are any consecutive NaN values or if the difference between consecutive years is greater than 10 or less than 0
    result = np.any((np.isnan(year_diff)) | (year_diff >= 10) | (year_diff < 0))
    
    return result

In [38]:
# Set up the main figure and axis
fig, ax = plt.subplots(figsize=(16, 8))
ax2 = ax.twinx()  # Create a secondary y-axis

# access legend objects automatically created from data
handles, labels = plt.gca().get_legend_handles_labels()

# Plot individual stake data
for stake_name, group_data in df_iceland_amb.groupby('stake'):
    if consecutive_nan(group_data.yr.values) or len(group_data.yr.values) <= 10:
        continue
    else: 
        ax.plot(group_data['yr'], group_data['ba_stratigraphic'], alpha=0.225, color='#4e8cd9', linewidth=0.9)

# Plot mean annual mass balance with standard deviation shading
mean_amb = ax.plot(mean_ba_per_year, linestyle='--', color='#ff7f0e', label='Mean Annual SMB', linewidth=3)
std_amb = ax.fill_between(mean_ba_per_year.index, 
                           mean_ba_per_year - std_ba_per_year, 
                           mean_ba_per_year + std_ba_per_year, 
                           color='orange', alpha=0.3, label='Std Annual SMB')

# Highlight significant events with scatter plots
ax.scatter(x=2010, y=mean_ba_per_year[2010], color='#eb10d1', s=150, marker='^', alpha=0.8) 
ax.scatter(x=2015, y=mean_ba_per_year[2015], color='#eb10d1', s=150, marker='8', alpha=0.6) 

# Get min and max year from ds dataset
min_year_ds = pd.Timestamp(ds.time.min().values).year
max_year_ds = pd.Timestamp(ds.time.max().values).year

# Plot temperature data on secondary y-axis
ax2.plot(np.linspace(min_year_ds, max_year_ds, len(temps)), temps, dashes=[25, 5, 10, 5], color='#f73b3b', linewidth=2.5, label='Mean t2m')

# Plot overall trendline
overall_trend = np.polyfit(mean_ba_per_year.index, mean_ba_per_year.values, 1)
overall_fit = np.poly1d(overall_trend)
x_fit_overall = np.linspace(mean_ba_per_year.index.min(), mean_ba_per_year.index.max())
ax.plot(x_fit_overall, overall_fit(x_fit_overall), linestyle='-.', marker='X', markevery=4, markersize=12, linewidth=2.5, 
        color='#4b9c8d', label=f'Mean Trendline (from {mean_ba_per_year.index.min()} to {mean_ba_per_year.index.max()})')

# Plot recent trendline
from_year = 2012
max_year = np.max(mean_ba_per_year.index)
recent_trend = np.polyfit(mean_ba_per_year.loc[from_year:max_year].index, mean_ba_per_year.loc[from_year:max_year].values, 1)
recent_fit = np.poly1d(recent_trend)
x_fit_recent = np.linspace(from_year, max_year)
ax.plot(x_fit_recent, recent_fit(x_fit_recent), linestyle='-.', marker='p', markevery=6, markersize=12, linewidth=2.5, 
        color='#9c4b75', label=f'Mean Trendline (from {from_year} to {max_year})')

# Combine legends from both axes
ax.plot(np.NaN, np.NaN, '-',  alpha=0.225, color='#4e8cd9', linewidth=0.9, label='Annual SMB per Stake')
handles1, labels1 = ax.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax.legend(handles=handles1 + handles2, labels=labels1 + labels2, loc='lower left')

# Set titles and labels
plt.title(f'Annual SMB for All Stakes and Mean Air Temperature from {np.min(mean_ba_per_year.index)} to {np.max(mean_ba_per_year.index)}', fontsize=17)
ax.set_xlabel('Year', fontsize=15)
ax.set_ylabel('Annual SMB [m w.e.]', fontsize=15)
ax2.set_ylabel('Mean Temperature [°C]', fontsize=15, rotation=-90, labelpad=17.5)

# Adjust tick parameters
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15, colors='#ff7f0e')
ax2.yaxis.set_tick_params(labelsize=15, colors='#f73b3b')

# Add grid, set limits, and layout adjustments
ax.set_xlim(left=np.min(mean_ba_per_year.index), right=np.max(mean_ba_per_year.index))
ax.set_ylim(bottom=-8, top=5)
ax2.set_ylim(bottom=-8, top=5)

# Color adjustments
ax.yaxis.label.set_color('#ff7f0e')     
ax2.yaxis.label.set_color('#f73b3b')    

# Annotations
ax.annotate("Eruption Eyjafjallajökull Volcano",
            xy=(2010, mean_ba_per_year[2010]-0.15), xycoords='data',
            xytext=(2012, mean_ba_per_year[2010]-3), textcoords='data', size=14, va="center", ha="left",
            arrowprops=dict(arrowstyle="->, head_width=0.4", connectionstyle="arc3,rad=-.5", color='black'), color='#eb10d1')
ax.annotate("Record High Avg. Temp. in 2015 Since 1880",
            xy=(2015, mean_ba_per_year[2015]+0.15), xycoords='data',
            xytext=(2013, mean_ba_per_year[2015]+3), textcoords='data', size=14, va="center", ha="right",
            arrowprops=dict(arrowstyle="->, head_width=0.4", connectionstyle="arc3,rad=-.5", color='black'), color='#eb10d1')

plt.grid(False)
ax.grid(False)
ax.yaxis.grid(color='gray', linestyle='-', alpha=0.3)

# Save and display the plot
if 'PLOTTING' in globals() and PLOTTING:
    plt.savefig(file_dir + f'plots/timeseries.svg', dpi=600, format='svg', bbox_inches='tight')
plt.show()
plt.close(fig)

### Create a Hypsometry Plot and Stake Distribution for the Glaciers in Iceland

In [23]:
# Read data
areas_rgiids = pd.read_csv(file_dir + 'files/areas_rgiids.csv')
joined_dfs = pd.merge(df_iceland, areas_rgiids, on='RGIId')

# Define elevation bins
elevation_bins = np.arange(0, 1800, 100)

# Aggregate area data by elevation bins
joined_dfs['elevation_bin'] = pd.cut(joined_dfs['elevation'], bins=elevation_bins, right=False)
area_distribution = joined_dfs.groupby('elevation_bin')['area'].sum()

# Calculate median and IQR
median_elevation = np.median(joined_dfs['elevation'])
lower_quartile = np.percentile(joined_dfs['elevation'], 25)
upper_quartile = np.percentile(joined_dfs['elevation'], 75)
iqr = upper_quartile - lower_quartile

# Aggregate stakes data by elevation bins
stakes_distribution = joined_dfs['elevation_bin'].value_counts().sort_index()

# Plotting
fig, ax1 = plt.subplots(figsize=(10, 8))

# Primary x-axis and bar plot for area distribution
ax1.barh(area_distribution.index.astype(str), area_distribution.values, color='#4e8cd9', alpha=0.7)
ax1.set_xlabel('Total Area [km²]', fontsize=12)
ax1.set_ylabel('Elevation [m]', fontsize=12)
ax1.set_title('Glacier Area and Stake Distribution by Elevation in Iceland', fontsize=15)
ax1.set_yticks(np.arange(len(area_distribution)))
ax1.set_yticklabels([f'{int(bin.left)}-{int(bin.right)} m' for bin in area_distribution.index])
ax1.axhline(y=median_elevation // 100, linestyle='--', color='#9c4b75', label=f'Median Elevation per Tot. Area: {median_elevation} m', linewidth=2)
ax1.axhline(y=(median_elevation + iqr) // 100, linestyle='-.', color='#4b9c8d', label=f'Median Elevation per Tot. Area + IQR', linewidth=2)
ax1.axhline(y=(median_elevation - iqr) // 100, linestyle='-.', color='#4b9c8d', label=f'Median Elevation per Tot. Area - IQR', linewidth=2)
ax1.legend()

# Secondary x-axis and histogram for stakes count
ax2 = ax1.twiny()
ax2.barh(stakes_distribution.index.astype(str), stakes_distribution.values, color='#9c4b75', alpha=0.3)
ax2.set_xlabel('Number of Stakes', fontsize=12)

ax1.xaxis.set_tick_params(labelsize=12, colors='#4e8cd9')
ax2.xaxis.set_tick_params(labelsize=12, colors='#9c4b75')

# Color adjustments
ax1.xaxis.label.set_color('#4e8cd9')     
ax2.xaxis.label.set_color('#9c4b75')  

# Adjust layout and grid
plt.tight_layout()
ax1.grid(True, axis='x', linestyle='--', linewidth=0.7)
ax1.yaxis.grid(False)
plt.grid(False)

if 'PLOTTING' in globals() and PLOTTING:
    plt.savefig(file_dir + f'plots/hypsometry_with_stakes.svg', dpi=600, format='svg', bbox_inches='tight')
plt.show()
plt.close(fig)